# Carga de información a la base de datos BBDD_Hoteles

En este documento realizamos la carga de los datos limpiados a la base de sql.

In [1]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import psycopg2 as ps

In [2]:
# Importamos el csv con la información de las reservas, los hoteles y los clientes
data = pd.read_csv("../data/reservas_hoteles_limpio.csv", parse_dates=['fecha_reserva', 'inicio_estancia', 'final_estancia'], dtype={'id_cliente': str, 'id_hotel': str})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    15000 non-null  datetime64[ns]
 7   inicio_estancia  15000 non-null  datetime64[ns]
 8   final_estancia   15000 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  object        
 10  nombre_hotel     15000 non-null  object        
 11  ciudad           15000 non-null  object        
 12  precio_noche     15000 non-null  float64       
 13  estrellas        15000 non-null  float64       
dtypes: bool(1), datetime64[ns](3), float64

In [3]:
data[["id_hotel", "nombre_hotel"]]

,id_hotel,nombre_hotel
0,1,Hotel Monte Verde
1,2,Hotel Brisas del Mar
2,3,Hotel Camino del Sol
3,4,Hotel Puerta del Cielo
4,5,Hotel Encanto Real
...,...,...
14995,28,ibis Madrid Alcobendas
14996,26,ibis Madrid Alcorcon Tresaguas
14997,27,ibis budget Madrid Aeropuerto
14998,23,ibis budget Madrid Centro las Ventas


In [13]:
# Importamos el csv con los eventos de Madrid sacados de la API
eventos = pd.read_csv("../data/eventos_madrid.csv", parse_dates=['fecha_inicio', 'fecha_fin'])
eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_evento  217 non-null    object        
 1   url_evento     217 non-null    object        
 2   codigo_postal  217 non-null    int64         
 3   direccion      201 non-null    object        
 4   horario        134 non-null    object        
 5   organizacion   202 non-null    object        
 6   fecha_inicio   217 non-null    datetime64[ns]
 7   fecha_fin      217 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 13.7+ KB


## Inicialización de la conexión con la BBDD y el cursor

In [4]:
# Creamos la conexión a la base de datos
# Vamos a crear una conexión a la base de datos.
conn = ps.connect(
    dbname = "BBDD_Hoteles", # base a la que nos queremos conectar
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432" # puerto en el que s eencuentra postgres
)

In [5]:
# Creamos un cursor el cual nos va a permitir ejecutar querys.
cur = conn.cursor()

In [6]:
# COmprobamos que la conexión está creada y conectada
cur.execute("SELECT version();")
cur.fetchone() 

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

## Inserción de datos

In [7]:
# COmprobamos que no haya valores duplicados en ambos dataframes
# data = data.drop_duplicates()
data.duplicated().sum()

np.int64(0)

### Tabla ciudad

In [18]:
data_insert_ciudad = ["Madrid"]

In [19]:
# Creamos la query de inserción de los datos de ciudad
insert_query_ciudad = """ 
                        INSERT INTO ciudad (nombre_ciudad)
                        VALUES (%s) 
"""

In [20]:
# Ejecutamos la query y le indicamos que solo tenemos una ciudad que es Madrid
cur.execute(insert_query_ciudad, data_insert_ciudad)
conn.commit()

In [11]:
# vamos a comprobar que se ha creado correctamente
query_ciudad = """
    SELECT * 
    FROM ciudad; 
"""
cur.execute(query_ciudad)
cur.fetchall()

[(1, 'Madrid')]

# NO VOLVER A EJECUTAR LA QUERY DE CIUDAD, ESTA YA INSERTADA

### Tabla eventos

In [8]:
# Sacamos el id de ciudad de la tabla de ciudad
cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad")
dictio_ciudad = dict(cur.fetchall())
dictio_ciudad

{'Madrid': 1}

In [23]:
# Creamos la lista que va a contener toda la información 
data_insert_eventos = []

# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in eventos.iterrows():
    nombre_evento = row["nombre_evento"]
    url_evento = row["url_evento"]
    codigo_postal = row["codigo_postal"]
    direccion = row["direccion"] if pd.notna(row["direccion"]) else None # tenemos valores nulos en esta columna, de manera que le indicamos, que si el elemento no es nulo nos lo coja, y si es nulo, nos incluya un None, ya que sql no acepta valores nulos que no sean None
    horario = row["horario"] if pd.notna(row["horario"]) else None
    fecha_inicio = row["fecha_inicio"]
    fecha_fin = row["fecha_fin"]
    organizacion = row["organizacion"] if pd.notna(row["organizacion"]) else None
    id_ciudad = dictio_ciudad.get("Madrid")

    # incluimos los valores en la lista
    data_insert_eventos.append((nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad))

In [24]:
# Creamos la query de inserción de los datos de eventos
insert_query_eventos = """ 
                        INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario,
                        fecha_inicio, fecha_fin,organizacion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [25]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_eventos, data_insert_eventos)
conn.commit()

In [7]:
# vamos a comprobar que se ha creado correctamente
query_eventos = """
    SELECT * 
    FROM eventos; 
"""
cur.execute(query_eventos)
cur.fetchmany(3)

[(1,
  '25º aniversario de la revista La Fragua',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=1ea220bed10d4910VgnVCM2000001f4a900aRCRD',
  28005,
  'CALLE SAN JUSTO 5',
  None,
  datetime.date(2025, 2, 24),
  datetime.date(2025, 3, 2),
  'Biblioteca Pública Municipal Iván de Vargas (Centro)',
  1),
 (2,
  '60 Premio Reina Sofía de Pintura y Escultura',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=34c5130393e15910VgnVCM2000001f4a900aRCRD',
  28009,
  'PASEO COLOMBIA 1',
  None,
  datetime.date(2025, 2, 27),
  datetime.date(2025, 3, 23),
  'Centro Cultural Casa de Vacas (Retiro)',
  1),
 (3,
  'A toda vela',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=59d5e7da8b822910VgnVCM1000001d4a900aRCRD',
  28045,
  'PLAZA LEGAZPI 8',
  None,
  datetime.date(2024, 10, 1),
  datetime.date(2025, 6, 1),
  'Matadero Ma

# NO VOLVER A EJECUTAR LA QUERY DE EVENTOS, ESTA YA INSERTADA

### Tabla hoteles

In [9]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'nombre_hotel', 'ciudad', 'precio_noche', 'estrellas'],
      dtype='object')

In [10]:
data_insert_hotels = []

for _, row in data.iterrows():
    id_hotel = row["id_hotel"]
    nombre_hotel = row["nombre_hotel"] 
    competencia = row["competencia"] 
    valoracion = row["estrellas"] 
    id_ciudad = dictio_ciudad.get("Madrid") 

    tupla_hotel = (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)

    if tupla_hotel not in data_insert_hotels:
        data_insert_hotels.append(tupla_hotel)

In [11]:
data_insert_hotels

[('1', 'Hotel Monte Verde', False, 3.1, 1),
 ('2', 'Hotel Brisas del Mar', False, 3.09, 1),
 ('3', 'Hotel Camino del Sol', False, 3.07, 1),
 ('4', 'Hotel Puerta del Cielo', False, 3.03, 1),
 ('5', 'Hotel Encanto Real', False, 3.03, 1),
 ('6', 'Palacio del Sol', False, 2.99, 1),
 ('7', 'Hotel Jardines del Rey', False, 2.94, 1),
 ('8', 'Hotel Las Estrellas', False, 2.91, 1),
 ('9', 'Gran Hotel Madrid', False, 3.07, 1),
 ('10', 'Hotel Torre Dorada', False, 2.92, 1),
 ('11', 'Hotel Palacio Imperial', False, 3.01, 1),
 ('12', 'Hotel Luz de Madrid', False, 3.05, 1),
 ('13', 'Hotel Los Almendros', False, 3.01, 1),
 ('14', 'Hotel Sol y Luna', False, 3.01, 1),
 ('15', 'Hotel Mirador Real', False, 2.98, 1),
 ('16', 'Hotel Rincón Sereno', False, 3.0, 1),
 ('17', 'Hotel Vista Alegre', False, 2.94, 1),
 ('18', 'Hotel Costa Azul', False, 3.12, 1),
 ('19', 'Hotel Maravilla Real', False, 2.98, 1),
 ('20', 'ibis Styles Madrid Prado', True, 4.7, 1),
 ('21', 'ibis budget Madrid Calle 30', True, 4.4, 1),


In [12]:
# Creamos la query de inserción de los datos de ciudad
insert_query_hoteles = """ 
                        INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s)
"""

In [39]:
conn.rollback()

In [13]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_hoteles, data_insert_hotels)
conn.commit()

In [14]:
# vamos a comprobar que se ha creado correctamente
query_hoteles = """
    SELECT * 
    FROM hoteles; 
"""
cur.execute(query_hoteles)
cur.fetchmany(3)

[('1', 'Hotel Monte Verde', False, 3.1, 1),
 ('2', 'Hotel Brisas del Mar', False, 3.09, 1),
 ('3', 'Hotel Camino del Sol', False, 3.07, 1)]

# HOTELES ESTA PERFECTO NO TOCAR MAS

### Tabla clientes

In [15]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'nombre_hotel', 'ciudad', 'precio_noche', 'estrellas'],
      dtype='object')

In [16]:
data_insert_clientes = []

for _, row in data.iterrows():

    id_cliente = row["id_cliente"]
    nombre = row["nombre"],
    apellido = row["apellido"],
    mail = row["mail"]

    tupla_cliente = (id_cliente, nombre, apellido, mail)

    if tupla_cliente not in data_insert_clientes:
        data_insert_clientes.append(tupla_cliente)

In [17]:
# Creamos la query de inserción de los datos de ciudad
insert_query_clientes = """ 
                        INSERT INTO clientes (id_cliente, nombre, apellido, mail)
                        VALUES (%s, %s, %s, %s)
"""

In [18]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_clientes, data_insert_clientes)
conn.commit()

In [19]:
# vamos a comprobar que se ha creado correctamente
query_clientes = """
    SELECT * 
    FROM clientes; 
"""
cur.execute(query_clientes)
cur.fetchmany(3)

[('1', 'Maite', 'Calatayud', 'maite.calatayud@example.com'),
 ('2', 'Tecla', 'Bonet', 'tecla.bonet@example.com'),
 ('3', 'Amílcar', 'Andrés', 'amílcar.andrés@example.com')]

### Tabla reservas

In [20]:
# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'nombre_hotel', 'ciudad', 'precio_noche', 'estrellas'],
      dtype='object')

In [21]:
# Sacamos el id de clientes de la tabla de clientes
cur.execute("SELECT nombre, id_cliente FROM clientes")
clientes = dict(cur.fetchall())
clientes

{'Maite': '13141',
 'Tecla': '14814',
 'Amílcar': '13479',
 'Joan': '14680',
 'Chelo': '12680',
 'Dolores': '13534',
 'Noé': '14144',
 'Omar': '14608',
 'Ágata': '14123',
 'Julie': '13797',
 'Javier': '10663',
 'Rita': '14765',
 'Ricardo': '14313',
 'Perlita': '14672',
 'Pedro': '14605',
 'Judith': '14238',
 'Francisco Javier': '14442',
 'Reynaldo': '14241',
 'Felicidad': '13027',
 'Pía': '14520',
 'Blas': '14336',
 'Claudia': '14842',
 'Fernanda': '13537',
 'Emilio': '14770',
 'Bárbara': '13760',
 'Emiliano': '14303',
 'Marisol': '14202',
 'Ezequiel': '14024',
 'Telmo': '14502',
 'Débora': '14653',
 'Darío': '14293',
 'Fausto': '14837',
 'Lorenza': '13933',
 'Anunciación': '14504',
 'María Belén': '13644',
 'Amaya': '14020',
 'Jose Angel': '14335',
 'Berta': '11535',
 'Salud': '14880',
 'Úrsula': '13529',
 'Sabas': '14723',
 'Asunción': '14667',
 'Jafet': '13640',
 'Adrián': '13868',
 'Benigno': '14660',
 'Jordán': '14083',
 'Sofía': '14576',
 'Ainara': '14286',
 'Segismundo': '14173'

In [22]:
# Sacamos el id de hotel de la tabla de hoteles
cur.execute("SELECT nombre_hotel, id_hotel FROM hoteles")
hoteles = dict(cur.fetchall())
hoteles

{'Hotel Monte Verde': '1',
 'Hotel Brisas del Mar': '2',
 'Hotel Camino del Sol': '3',
 'Hotel Puerta del Cielo': '4',
 'Hotel Encanto Real': '5',
 'Palacio del Sol': '6',
 'Hotel Jardines del Rey': '7',
 'Hotel Las Estrellas': '8',
 'Gran Hotel Madrid': '9',
 'Hotel Torre Dorada': '10',
 'Hotel Palacio Imperial': '11',
 'Hotel Luz de Madrid': '12',
 'Hotel Los Almendros': '13',
 'Hotel Sol y Luna': '14',
 'Hotel Mirador Real': '15',
 'Hotel Rincón Sereno': '16',
 'Hotel Vista Alegre': '17',
 'Hotel Costa Azul': '18',
 'Hotel Maravilla Real': '19',
 'ibis Styles Madrid Prado': '20',
 'ibis budget Madrid Calle 30': '21',
 'ibis Madrid Centro las Ventas': '22',
 'ibis budget Madrid Centro las Ventas': '23',
 'ibis budget Madrid Vallecas': '24',
 'ibis Madrid Aeropuerto Barajas': '25',
 'ibis Madrid Alcorcon Tresaguas': '26',
 'ibis budget Madrid Aeropuerto': '27',
 'ibis Madrid Alcobendas': '28',
 'ibis budget Madrid Alcorcon Móstoles': '29'}

In [43]:
data_insert_reservas = []

for _, row in data.iterrows():
    id_reserva = row["id_reserva"]
    fecha_reserva = row["fecha_reserva"]
    inicio_estancia = row["inicio_estancia"]
    final_estancia = row["final_estancia"]
    precio_noche = data["precio_noche"]
    id_cliente = clientes.get(row["id_cliente"])
    id_hotel = hoteles.get(row["id_hotel"])

    data_insert_reservas.append((id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel))

In [44]:
data_insert_reservas

[('40c4cb55-d1f5-407b-832f-4756b8ff77b4',
  Timestamp('2025-02-09 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-02 00:00:00'),
  0        276.612381
  1        275.225921
  2        269.998444
  3        280.151243
  4        278.243996
              ...    
  14995     85.000000
  14996     90.000000
  14997     88.000000
  14998    119.000000
  14999    110.000000
  Name: precio_noche, Length: 15000, dtype: float64,
  None,
  None),
 ('f2ce8df5-7844-43e1-8c0f-97ce7a208f21',
  Timestamp('2025-02-08 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-02 00:00:00'),
  0        276.612381
  1        275.225921
  2        269.998444
  3        280.151243
  4        278.243996
              ...    
  14995     85.000000
  14996     90.000000
  14997     88.000000
  14998    119.000000
  14999    110.000000
  Name: precio_noche, Length: 15000, dtype: float64,
  None,
  None),
 ('57d4515a-447d-4067-afcb-9bdcf4e4e915',
  Timestamp('2025-02-02 00:00:00')

In [45]:
conn.rollback()

In [46]:
# Creamos la query de inserción de los datos de ciudad
insert_query_reservas = """ 
                        INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [47]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_reservas, data_insert_reservas)
conn.commit()

ProgrammingError: can't adapt type 'Series'

## Cierre de la conexión

In [13]:
# una vez hemos terminado de trabajar es necesario cerrar la conexión y el cursor.
cur.close()
conn.close()